## Naivni Bayesov klasifikator
za napovedovanje cene rabljenega avtomobila.

In [35]:
import pandas as pd

avti = pd.read_csv("../podatki/rabljeni_avtomobili.csv")

# zvezne parametre zaokrožimo
avti.cena = avti.cena.round(-3)
avti.kilovati = avti.kilovati.round(-1)
avti.stevilo_kilometrov = avti.stevilo_kilometrov.round(-5)

Avtomobile razdelimo na učno in testno množico.

In [36]:
import numpy as np
maska = np.random.rand(len(avti)) < 0.9
parametri = ["ime_znamke", "id_modela", "menjalnik", "vrsta_motorja", "tip_modela", "kilovati", "leto_izdelave", "stevilo_kilometrov"]
učni_avti = avti[maska][parametri+["cena"]].dropna()
testni_avti = avti[~maska][parametri+["cena"]].dropna()

In [37]:
pojavitve_cen = učni_avti.groupby("cena").size()
verjetnosti_cen = pojavitve_cen / len(učni_avti)

$p_i$ ... i-ti parameter  
$c$ ... cena
$$P(P_i = p_i \mid C = c)$$

In [38]:
pogojne_verjetnosti_parametrov = pd.concat(
    [
        (učni_avti.groupby(["cena", parameter]).size() / pojavitve_cen)
        .unstack().transpose().fillna(1/len(učni_avti))
        for parameter
        in parametri
    ],
    keys=parametri
)

In [39]:
def napovej_ceno(avtomobil: pd.Series):
    indeksi = set(zip(avtomobil.index, avtomobil))
    verjetne_cene = pogojne_verjetnosti_parametrov[
        pogojne_verjetnosti_parametrov.index.isin(indeksi)
    ].product(numeric_only=True)*verjetnosti_cen
    verjetne_cene /= verjetne_cene.sum()
    return (verjetne_cene.index * verjetne_cene).sum().round(-2)

In [40]:
testni_avti["napovedana_cena"] = testni_avti.apply(napovej_ceno, axis=1)

Povprečna napaka napovedi

In [41]:
RMSE = (((testni_avti.napovedana_cena-testni_avti.cena)**2).sum()/len(testni_avti))**0.5
RMSE

7374.02247643667

In [42]:
testni_avti.sample(10)

,ime_znamke,id_modela,menjalnik,vrsta_motorja,tip_modela,kilovati,leto_izdelave,stevilo_kilometrov,cena,napovedana_cena
6122,Fiat,791.0,ROČNI,BENCIN,HATCHBACK,50.0,2016.0,100000.0,10000.0,9000.0
20238,Volkswagen,2638.0,AVTOMATSKI,DIEZEL,KARAVAN,100.0,2008.0,300000.0,2000.0,6100.0
1677,Audi,147.0,AVTOMATSKI,DIEZEL,HATCHBACK,110.0,2013.0,200000.0,16000.0,15600.0
7433,Ford,1010.0,ROČNI,BENCIN,COUPÉ,120.0,1997.0,200000.0,8000.0,8800.0
22404,Volkswagen,2671.0,ROČNI,DIEZEL,KOMBI,70.0,2011.0,200000.0,12000.0,7700.0
273,Aston Martin,102.0,AVTOMATSKI,BENCIN,COUPÉ,470.0,2021.0,0.0,249000.0,154000.0
9094,Lancia,1416.0,ROČNI,BENCIN,HATCHBACK,140.0,1996.0,200000.0,5000.0,1700.0
19709,Volkswagen,2630.0,ROČNI,DIEZEL,KARAVAN,80.0,2013.0,200000.0,9000.0,11700.0
2747,Audi,140.0,ROČNI,DIEZEL,SUV,80.0,2020.0,100000.0,24000.0,23000.0
6231,Fiat,879.0,ROČNI,DIEZEL,HATCHBACK,90.0,2011.0,200000.0,3000.0,3700.0
